#  Pacific Value Testing


Funding elements for a creative work and get the info from IOC and other partners that Paul has identified.


![q1](./q1.png)

![q2](./q2.png)

In [13]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
# import dask, boto3
# import dask.dataframe as dd
import numpy as np
import json
import geopandas
import matplotlib.pyplot as plt
import shapely
# import kglab as kg

sparqlep = "https://ts.collaborium.io/blazegraph/namespace/development/sparql"

In [14]:
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [15]:
rq1 = """prefix prov: <http://www.w3.org/ns/prov#>
PREFIX con: <http://www.ontotext.com/connectors/lucene#>
PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX schemaold: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?pred
WHERE
{
   ?gs a schema:GeoShape .
   ?gs ?pred ?onj .             
}


"""
df1 = get_sparql_dataframe(sparqlep, rq1)
# dfc['pCount'] = dfc["pCount"].astype(int) # convert count to int
# dfc.set_index('p', inplace=True)
# dfc_sorted = dfc.sort_values('pCount', ascending=False)
df1.head(10)

,pred
0,https://schema.org/polygon
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type


In [16]:
rq2 = """prefix prov: <http://www.w3.org/ns/prov#>
PREFIX con: <http://www.ontotext.com/connectors/lucene#>
PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX schemaold: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?domain  ( COUNT(?domain) as ?count )
WHERE
{
  graph ?g {
  ?gs a schema:Place .
      }
           ?sp prov:generated ?g  .
           ?sp prov:used ?used .
           ?used prov:hadMember ?hm .
           ?hm prov:wasAttributedTo ?wat .
           ?wat rdf:name ?orgname .
           ?wat rdfs:seeAlso ?domain
}
 GROUP BY ?domain
  ORDER BY DESC(?count)
      
"""
df2 = get_sparql_dataframe(sparqlep, rq2)
# dfc['pCount'] = dfc["pCount"].astype(int) # convert count to int
# dfc.set_index('p', inplace=True)
# dfc_sorted = dfc.sort_values('pCount', ascending=False)
df2.head(10)

,domain,count
0,https://obis.org,7363
1,https://edmo.seadatanet.org,4279
2,https://oceanexpert.org/,2715
3,https://infohub.eurocean.net/data/events,2711
4,https://marinetraining.eu/,309


In [17]:
rq3 = """ prefix prov: <http://www.w3.org/ns/prov#>
PREFIX con: <http://www.ontotext.com/connectors/lucene#>
PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX schemaold: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?domain  ( COUNT(?domain) as ?count )
WHERE
{
  graph ?g {
   ?s schema:identifier ?id .
  ?id schema:value ?value.
  ?id schema:url ?url
      }
                  ?sp prov:generated ?g  .
           ?sp prov:used ?used .
           ?used prov:hadMember ?hm .
           ?hm prov:wasAttributedTo ?wat .
           ?wat rdf:name ?orgname .
           ?wat rdfs:seeAlso ?domain
}
 GROUP BY ?domain
  ORDER BY DESC(?count)
      

"""
df3 = get_sparql_dataframe(sparqlep, rq3)
# dfc['pCount'] = dfc["pCount"].astype(int) # convert count to int
# dfc.set_index('p', inplace=True)
# dfc_sorted = dfc.sort_values('pCount', ascending=False)
df3.head(10)

,domain,count
0,https://aquadocs.org,39116
1,https://oih.oceanbestpractices.org,1420
2,https://obis.org,1351


In [19]:
rq4 = """prefix prov: <http://www.w3.org/ns/prov#>
PREFIX con: <http://www.ontotext.com/connectors/lucene#>
PREFIX luc: <http://www.ontotext.com/owlim/lucene#>
PREFIX con-inst: <http://www.ontotext.com/connectors/lucene/instance#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX schemaold: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?p ?orgname
WHERE
{
  graph ?g {
  ?gs a schema:Place .
  ?gs ?p ?o .
      }
           ?sp prov:generated ?g  .
           ?sp prov:used ?used .
           ?used prov:hadMember ?hm .
           ?hm prov:wasAttributedTo ?wat .
           ?wat rdf:name ?orgname .
           ?wat rdfs:seeAlso ?domain
}

      

"""
df4 = get_sparql_dataframe(sparqlep, rq4)
# dfc['pCount'] = dfc["pCount"].astype(int) # convert count to int
# dfc.set_index('p', inplace=True)
# dfc_sorted = dfc.sort_values('pCount', ascending=False)
df4.head(10)

,p,orgname
0,https://schema.org/name,Marine Training EU
1,https://schema.org/address,Marine Training EU
2,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Marine Training EU
3,https://schema.org/additionalProperty,Ocean Biodiversity Information System
4,https://schema.org/geo,Ocean Biodiversity Information System
5,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,Ocean Biodiversity Information System
6,https://schema.org/address,OceanExpert UNESCO/IOC Project Office for IODE
7,https://schema.org/address,EurOcean Events
8,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,OceanExpert UNESCO/IOC Project Office for IODE
9,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,EurOcean Events
